In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import tensorflow as tf
import os
from tqdm import tqdm
seed = 1378
np.random.seed(seed)

# فرض می‌کنیم شما یک تصویر چهره 256x256 و نقاط مربوط به آن در همان مقیاس دارید.
def gaussian_probability(distance, sigma=15):
        return np.exp(-(distance ** 2) / (2 * sigma ** 2))
def process_face_image(face_img, face_landmarks):
    # Randomly selecting a 40x40 patch in the 256x256 face image
    x = np.random.randint(0, 256 - 40)
    y = np.random.randint(0, 256 - 40)
    patch = face_img[y:y + 40, x:x + 40]

    # Calculating the center of the patch
    patch_center = np.array([x + 20, y + 20])

    # Defining a Gaussian function to calculate probability based on distance


    # Calculate the probability for each face landmark based on its distance from patch center
    probabilities = np.zeros(len(face_landmarks))
    for i, point in enumerate(face_landmarks):
        distance = np.linalg.norm(np.array(point) - patch_center)
        probabilities[i] = gaussian_probability(distance)

    # Normalizing the probabilities to sum to 1
    probabilities /= np.sum(probabilities)

    return probabilities , patch

# فرض کنید 'face_img' و 'face_landmarks' تصویر چهره‌ی 256x256 و نقاط چهره‌ی اسکیل‌شده را دارند.
# برای تست، می‌توانیم از تصویر و نقاط نمونه استفاده کنیم.
# مثلاً یک تصویر سفید و نقاط تصادفی برای چهره

# ایجاد یک تصویر سفید برای تست
# face_img = cv2.imread('/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_live_10_HR_1/cropped_0025.jpg')


# # ایجاد نقاط چهره‌ی تصادفی (در اندازه‌ی 256x256) برای تست
# face_landmarks = np.load('/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/casia_test_live_10_HR_1/cropped_0025.npy',allow_pickle=True).item()['landmark']


# # فراخوانی تابع برای پردازش
# P = process_face_image(face_img, face_landmarks)


In [2]:
from tensorflow.keras.applications import DenseNet121
kernel_size = 20



model = DenseNet121(input_shape=(kernel_size*2,kernel_size*2,3),weights=None, include_top=False)

# Choose the desired low-level feature layers
low_level_layers = [
    model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
    model.get_layer('conv3_block12_concat')# Output of the last layer in the second dense block
]
# low_level_layers = [
#     model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
#     model.get_layer('conv3_block12_concat'), 
#     model.get_layer('conv4_block24_concat')# Output of the last layer in the second dense block
# ]

# Create a new model with the desired low-level feature layers as outputs
feature_extractor = tf.keras.Model(inputs=model.input, outputs=[layer.output for layer in low_level_layers])
feature_extractor.load_weights("/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.h5")
feature_extractor.trainable=False

2024-09-09 20:46:05.986835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-09 20:46:05.992257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-09 20:46:05.993107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-09 20:46:05.994278: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:

#input_image = tf.random.uniform((10,40,40,3))
def DenseNet_feature_extractor(input_image) :
    #input_image = tf.keras.preprocessing.image.img_to_array(input_image)
    #print("input shape", input_image.shape)
    input_image = tf.cast(input_image, tf.float32)
    input_image = tf.keras.applications.densenet.preprocess_input(input_image)
    #input_image = tf.expand_dims(input_image, axis=0)

    feature_extracted = feature_extractor.predict(input_image,verbose=False)
    #print(feature_extracted[0].shape,feature_extracted[1].shape)
    global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[0])
    flatten = tf.keras.layers.Flatten()(global_pooling)
    total_feature =flatten
    for layer_cnt in range(1,2) : 
        global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[layer_cnt])
        flatten = tf.keras.layers.Flatten()(global_pooling)
        total_feature = tf.concat([flatten,total_feature],1)
    return total_feature

In [4]:
MSU_path = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/*/cropped*.pt")

In [5]:
for graph_path in tqdm(MSU_path, desc="Processing faces"):
    # face_path = os.path.dirname(graph_path) + "/" + "org_" + os.path.basename(graph_path[:-6]).split("_")[1] + "jpg"
    landmark_path = graph_path[:-7] + ".npy"
    
    # Load the face image
    face = cv2.imread(graph_path[:-3])
    
    # Load the landmarks
    landmark = np.load(landmark_path, allow_pickle=True).item()['landmark']
    
    # Process the face image and landmark to get probability and face patch
    prob, face_patch = process_face_image(face, landmark)
    
    # Convert probability to tensor    
    # Convert face patch to RGB and expand dimensions
    face_patch = cv2.cvtColor(face_patch, cv2.COLOR_BGR2RGB)
    face_patch = np.expand_dims(face_patch, axis=0)
    
    # Extract features using DenseNet
    face_patch_feature = DenseNet_feature_extractor(face_patch).numpy()
    #print(os.path.dirname(graph_path[:-7])+"/"+"patch_feature_"+os.path.basename(graph_path[:-7]) + ".npy")
    np.save(os.path.dirname(graph_path[:-7])+"/"+"patch_feature_1_"+os.path.basename(graph_path[:-7]) + ".npy",
           face_patch_feature)
    np.save(os.path.dirname(graph_path[:-7])+"/"+"Prob_1_"+os.path.basename(graph_path[:-7]) + ".npy",prob)

Processing faces:   0%|                                                                                            | 0/15549 [00:00<?, ?it/s]2024-09-09 20:46:08.626285: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-09-09 20:46:09.148018: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-09 20:46:09.148464: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-09 20:46:09.148477: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-09-09 20:46:09.149050: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-09 20:46:09.149092: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Mod

In [8]:
MSU_path = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/*/cropped*.pt")

In [9]:
for graph_path in tqdm(MSU_path, desc="Processing faces"):
    # face_path = os.path.dirname(graph_path) + "/" + "org_" + os.path.basename(graph_path[:-6]).split("_")[1] + "jpg"
    landmark_path = graph_path[:-7] + ".npy"
    
    # Load the face image
    face = cv2.imread(graph_path[:-3])
    
    # Load the landmarks
    landmark = np.load(landmark_path, allow_pickle=True).item()['landmark']
    
    # Process the face image and landmark to get probability and face patch
    prob, face_patch = process_face_image(face, landmark)
    
    # Convert probability to tensor    
    # Convert face patch to RGB and expand dimensions
    face_patch = cv2.cvtColor(face_patch, cv2.COLOR_BGR2RGB)
    face_patch = np.expand_dims(face_patch, axis=0)
    
    # Extract features using DenseNet
    face_patch_feature = DenseNet_feature_extractor(face_patch).numpy()
    #print(os.path.dirname(graph_path[:-7])+"/"+"patch_feature_"+os.path.basename(graph_path[:-7]) + ".npy")
    np.save(os.path.dirname(graph_path[:-7])+"/"+"patch_feature_1_"+os.path.basename(graph_path[:-7]) + ".npy",
           face_patch_feature)
    np.save(os.path.dirname(graph_path[:-7])+"/"+"Prob_1_"+os.path.basename(graph_path[:-7]) + ".npy",prob)

Processing faces: 100%|████████████████████████████████████████████████████████████████████████████████| 61989/61989 [35:25<00:00, 29.16it/s]
